In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [2]:
dirpath = Path("../data")
dataset = "unhealthy"
filepath = dirpath / dataset / "test.csv"
df = pd.read_csv(filepath)
df.shape

(20112, 16)

In [3]:
def get_columns(dataset):
    if dataset == "go_emo":
        return sorted([
            'admiration','amusement', 'anger', 'annoyance', 'approval', 'caring',
            'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
            'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
            'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
            'remorse', 'sadness', 'surprise', 'neutral'
        ]), "rater_id", "text"
    if dataset == "unhealthy":
        return sorted([
            "antagonize", "condescending" , "dismissive", "generalisation",
            "generalisation_unfair", "healthy", "hostile", "sarcastic"
        ]), "_worker_id", "comment"
    if dataset == "docanno":
        return sorted([
            'inspiring', 'interesting', 'offensive_to_someone', 'negative',
            'offensive_to_me', 'political', 'positive', 'sadness', 'calm',
            'fear', 'compassion', 'disgust', 'vulgar', 'surprise', 'embarrasing',
            'anger', 'understandable', 'ironic', 'need_more_information',
            'happiness', 'delight', 'funny_to_someone', 'funny_to_me'
        ]), "user_id", "text"
    return None

In [4]:
label_columns, annotator_column, text_column = get_columns(dataset)
num_shots = 2

In [5]:
def parse_annotation_to_text(sample):
    response = ", ".join([label for label in label_columns if sample[label] == 1])
    return response

In [6]:
def get_examples(df, sample, num_shots=2):
    annotator_texts = df[df[annotator_column] == sample[annotator_column]].drop(sample["index"])
    if annotator_texts.shape[0] < num_shots:
        examples = annotator_texts.sample(frac=1)
    else:
        examples = annotator_texts.sample(num_shots)
    extracted_return = []
    for idx, ex in examples.iterrows():
        extracted_return.append(ex[text_column])
        extracted_return.append(parse_annotation_to_text(ex))
    while len(extracted_return) < 2*num_shots:
        extracted_return.append(None)
    return extracted_return

In [7]:
tqdm.pandas()
new_columns = sum([[f"example{i+1}", f"example{i+1}_response"] for i in range(num_shots)], start=[])
df[new_columns] = df.reset_index().progress_apply(lambda sample: get_examples(df, sample, 2), axis=1, result_type="expand")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20112/20112 [00:10<00:00, 1844.07it/s]


In [8]:
df.head()

,_unit_id,comment,_trust,_worker_id,antagonize,condescending,dismissive,generalisation,generalisation_unfair,healthy,hostile,sarcastic,example1,example1_response,example2,example2_response
0,2.327213e+09,"No, Olivia Chow has nothing to do with David M...",0.9667,18960682,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,"Instead of ?xe, xem and xyr?, why not 'it and ...",healthy,Britain which has a conservative government an...,healthy
1,1.739445e+09,"Comonsense, he disqualified himself when he fl...",0.9929,44022448,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Some people call people like that Psychopaths.,healthy,Absolutely spot on. But don't expect the flogg...,healthy
2,1.800633e+09,"Thanks for your considered opinion, so tempera...",0.9919,44083576,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"Israel is defending its continued occupation, ...",healthy,The native populations of North America know w...,healthy
3,1.739443e+09,When you start worrying about the 'human right...,0.9500,44556582,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,ya buddy... its those cats.... its not the pol...,"healthy, sarcastic",M.The media has the power. How much of the wor...,healthy
4,1.739442e+09,Like watching a class c Hollywood production t...,0.9929,44022448,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,http://www.politifact.com/truth-o-meter/articl...,healthy,"Down on Queens Quay, we have a fake 'fire call...",healthy


In [9]:
df.to_csv(filepath, index=False)